# Create summary tables for CIS-PD curation paper

# Next action
- clean up and consolidate code

In [33]:
# Importing the Libraries
import os
import platform
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import pathlib
import pickle
import time
import re
import copy
%matplotlib inline

# Load and check codelist and data dictionary
- codelist doesn't have form 508 or 238, only datadict does

In [24]:
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\codelist.sas7bdat'
codelist = pd.read_sas(filename)
codelist.columns = codelist.columns.str.replace('z', '')
codelist = codelist.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\datadictionary.sas7bdat'
datadict = pd.read_sas(filename, format = 'sas7bdat', encoding='iso-8859-1')
datadict.columns = datadict.columns.str.replace('z', '')

# Check updrs forms for parts 1,2,4 and 3

In [34]:
# load files
path = r'Y:\CIS-PD MUSC\decoded_forms'
data124 = pd.read_hdf(os.path.join(path,'updrs_124.h5'))
data3 = pd.read_hdf(os.path.join(path,'updrs_part3.h5'))

In [14]:
data124.columns.values

array(['Subject ID', 'SiteID', 'Visit', 'FormDate',
       'Primary source of information', '1.1', '1.2', '1.3', '1.4', '1.5',
       '1.6', 'Who is filling out this questionnaire', '1.7', '1.8',
       '1.9', '1.10', '1.11', '1.12', '1.13', '2.1', '2.2', '2.3', '2.4',
       '2.5', '2.6', '2.7', '2.8', '2.9', '2.10', '2.11', '2.12', '2.13',
       '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', 'DataCollected'],
      dtype=object)

In [95]:
data124.columns.values

array(['Subject ID', 'SiteID', 'Visit', 'FormDate',
       'Primary source of information', '1.1', '1.2', '1.3', '1.4', '1.5',
       '1.6', 'Who is filling out this questionnaire', '1.7', '1.8',
       '1.9', '1.10', '1.11', '1.12', '1.13', '2.1', '2.2', '2.3', '2.4',
       '2.5', '2.6', '2.7', '2.8', '2.9', '2.10', '2.11', '2.12', '2.13',
       '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', 'DataCollected'],
      dtype=object)

# Create Table for parts 1, 2, and 4

In [100]:
# get questions by slicing list for parts 1,2,4, subtract part 3
list = datadict.loc[datadict.FormID == 238.0]['FieldNm'].tolist()
qlist = list[:30] + list[70:-1]

In [122]:
name1 = ['One of: (\'Patient\', \'Caregiver\', \'Patient and Caregiver in Equal Proportion\')']
name2 = ['One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)']
variabledetails = ['Subject ID', 'SiteID', 'Visit', 'FormDate'] + \
                    name1 + name2*6 + name1 + name2*26 + ['One of: (\'no\',\'yes\')']

tabledata124 = {
'Question': ['Subject ID', 'SiteID', 'Visit',
             'Date of assessment',
             'Primary source of information',
             '1.1 Cognitive Impairment',
             '1.2 Hallucinations and Psychosis',
             '1.3 Depressed Mood',
             '1.4 Anxious Mood',
             '1.5 Apathy',
             '1.6 Features of Dopamine Dysregulation Syndrome',
             'Who is filling out this questionnaire',
             '1.7 Sleep Problems',
             '1.8 Daytime Sleepiness',
             '1.9 Pain and Other Sensations',
             '1.10 Urinary Problems',
             '1.11 Constipation Problems',
             '1.12 Light Headedness on Standing',
             '1.13 Fatigue',
             '2.1 Speech',
             '2.2 Saliva & Drooling',
             '2.3 Chewing and Swallowing',
             '2.4 Eating Tasks',
             '2.5 Dressing',
             '2.6 Hygiene',
             '2.7 Handwriting',
             '2.8 Doing Hobbies and Other Activities',
             '2.9 Turning in Bed',
             '2.10 Tremor',
             '2.11 Getting Out of Bed, a Car, or a Deep Chair',
             '2.12 Walking and Balance',
             '2.13 Freezing',
             '4.1 Time Spent With Dyskinesias',
             '4.2 Functional Impact of Dyskinesias',
             '4.3 Time Spent in the Off State',
             '4.4 Functional Impact of Fluctuations',
             '4.5 Complexity of Motor Fluctuations',
             '4.6 Painful Off-State Dystonia',
             'Data Collected'],
'Variable Name': data124.columns.values.tolist(),
'Variable details': variabledetails
}
df124 = pd.DataFrame(tabledata124)

# save table as csv file
tablepath = r'Y:\CIS-PD Study\MJFF Curation\tables'
filename = os.path.join(tablepath, 'tablepart124.csv')
with open(filename,'wb') as f:
    df124.to_csv(filename, sep=',')

# Create Table for part 3
Notes:
why does data3 have 
- ParticipantState and 3b - same
- What's the difference b/n UTC and FormTime?


In [94]:
data3.columns.values#.tolist()

array(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'UTC', 'FormTime',
       'ParticipantState', 'Hours since last PD medication dose', '3a',
       '3b', '3c', '3C1', '3.1', '3.2', '3.3 Neck',
       '3.3 Right Upper Extremity', '3.3 Left Upper Extremity',
       '3.3 Right Lower Extremity', '3.3 Left Lower Extremity',
       '3.4 Right Hand', '3.4 Left Hand', '3.5 Right Hand',
       '3.5 Left Hand', '3.6 Right Hand', '3.6 Left Hand',
       '3.7 Right Foot', '3.7 Left Foot', '3.8 Right Leg', '3.8 Left Leg',
       '3.9', '3.10', '3.11', '3.12', '3.13', '3.14', '3.15 Right Hand',
       '3.15 Left Hand', '3.16 Right Hand', '3.16 Left Hand',
       '3.17 Right Upper Extremity', '3.17 Left Upper Extremity',
       '3.17 Right Lower Extremity', '3.17 Left Lower Extremity',
       '3.17 Lip-Jaw', '3.18', '3.19A', '3.19B', '3.20', 'DataCollected'],
      dtype=object)

In [87]:
tabledata = {
'Question': ['Subject ID', 'SiteID', 'Visit', 'Date of assessment',
             'Time of assessment', 'FormTime',
             # repeated items with 3b - 'ParticipantState', 'Hours since last PD medication dose'???
             '3b Mark the patient\'s clinical state',
             'Hours since last PD medication dose',
             # 3a, 3b, 3c, 3C1
             '3a Is the patient on medication for treating the symptoms of Parkinson\'s Disease?',
             '3b Mark the patient\'s clinical state', 
             '3c Is the patient on Levodopa?',
             '3.C1 Minutes since last levodopa dose',
             '3.1 Speech',
             '3.2 Facial Expression',
             '3.3 Rigidity - Neck',
             '3.3 Rigidity - Right Upper Extremity',
             '3.3 Rigidity - Left Upper Extremity',
             '3.3 Rigidity - Right Lower Extremity',
             '3.3 Rigidity - Left Lower Extremity',
             '3.4 Finger Tapping - Right Hand',
             '3.4 Finger Tapping - Left Hand',
             '3.5 Hand Movements - Right Hand',
             '3.5 Hand Movements - Left Hand',
             '3.6 Pronation-Supination Movements of Hands - Right Hand',
             '3.6 Pronation-Supination Movements of Hands - Left Hand',
             '3.7 Toe Tapping - Right Foot',
             '3.7 Toe Tapping - Left Foot',
             '3.8 Leg Agility - Right Leg',
             '3.8 Leg Agility - Left Leg',
             '3.9 Arising From Chair',
             '3.10 Gait',
             '3.11 Freezing of Gait',
             '3.12 Postural Stability',
             '3.13 Posture',
             '3.14 Global Spontaneity of Movement Body Bradykinesia',
             '3.15 Postural Tremor of the Hands - Right Hand',
             '3.15 Postural Tremor of the Hands - Left Hand',
             '3.16 Kinetic Tremor of the Hands - Right Hand',
             '3.16 Kinetic Tremor of the Hands - Left Hand',
             '3.17 Rest Tremor Amplitude - Right Upper Extremity',
             '3.17 Rest Tremor Amplitude - Left Upper Extremity',
             '3.17 Rest Tremor Amplitude - Right Lower Extremity',
             '3.17 Rest Tremor Amplitude - Left Lower Extremity',
             '3.17 Rest Tremor Amplitude - Lip/Jaw',
             '3.18 Constancy of Rest Tremor',
             '3.19 A Were dyskinesias (chorea or dystonia) present during examination?',
             '3.19 B Did these movements interfere with your ratings?',
             '3.20 Hoen and Yahr Stage',
             'Data collected'],
'Variable Name': data3.columns.values.tolist(),
'Variable details': ['Subject ID', 'SiteID', 'Visit', 'Date of assessment',
                     'Datetime', 'FormTime',
                     # repeated items with 3b - 'ParticipantState', 'Hours since last PD medication dose'???
                     'One of: (\'on\',\'off\')',
                     'Integer',
                     # 3a, 3b, 3c, 3C1
                     'One of: (\'no\',\'yes\')',
                     'One of: (\'on\',\'off\')', 
                     'One of: (\'no\',\'yes\')',
                     'Integer',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'Normal\',\' Slight\',\' Mild\',\' Moderate\',\' Severe\') mapping to (0, 1, 2, 3, 4)',
                     'One of: (\'no\',\'yes\')',
                     'One of: (\'no\',\'yes\')',
                     'One of: (0, 1, 2, 3, 4, 5)',
                     'One of: (\'no\',\'yes\')']
}
df = pd.DataFrame(tabledata)
# save as csv file to create table

In [ ]:
df

In [90]:
# save table as csv file
tablepath = r'Y:\CIS-PD Study\MJFF Curation\tables'
filename = os.path.join(tablepath, 'tablepart3.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

# Create Demographics table

In [4]:
path = r'Y:\CIS-PD MUSC\decoded_forms'

In [8]:
# items: gender, race, ethnicity, age
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\subjenrollment.sas7bdat'
subjenroll = pd.read_sas(filename, format = 'sas7bdat', encoding='iso-8859-1')
subjenroll.columns = subjenroll.columns.str.replace('z', '')
print(subjenroll.head(5))

   SubjectCode FoxInsightID  SiteID  ConsentBeforeEnrollment  Gender  Race  \
0       1000.0     cisuaba1  1313.0                      1.0     1.0   5.0   
1       1001.0     cisuabb2  1313.0                      1.0     1.0   5.0   
2       1002.0     cisuabc3  1313.0                      1.0     1.0   5.0   
3       1003.0     cisuabd4  1313.0                      1.0     2.0   3.0   
4       1004.0      cisnwa1  1332.0                      1.0     1.0   5.0   

   Ethnicity   Age  ConsentDt EnrollmentDt  
0        2.0  63.0 2017-06-15   2017-06-15  
1        2.0  64.0 2017-06-15   2017-06-15  
2        2.0  51.0 2017-06-16   2017-06-16  
3        2.0  60.0 2017-06-19   2017-06-19  
4        2.0  52.0 2017-06-21   2017-06-21  


In [117]:
subjenroll.head(20)

,SubjectCode,FoxInsightID,SiteID,ConsentBeforeEnrollment,Gender,Race,Ethnicity,Age,ConsentDt,EnrollmentDt
0,1000.0,cisuaba1,1313.0,1.0,1.0,5.0,2.0,63.0,2017-06-15,2017-06-15
1,1001.0,cisuabb2,1313.0,1.0,1.0,5.0,2.0,64.0,2017-06-15,2017-06-15
2,1002.0,cisuabc3,1313.0,1.0,1.0,5.0,2.0,51.0,2017-06-16,2017-06-16
3,1003.0,cisuabd4,1313.0,1.0,2.0,3.0,2.0,60.0,2017-06-19,2017-06-19
4,1004.0,cisnwa1,1332.0,1.0,1.0,5.0,2.0,52.0,2017-06-21,2017-06-21
5,1005.0,cisuabe5,1313.0,1.0,1.0,2.0,2.0,52.0,2017-06-21,2017-06-21
6,1006.0,cisroa1,1396.0,1.0,1.0,5.0,2.0,71.0,2017-06-21,2017-06-21
7,1007.0,cisuabf6,1313.0,1.0,1.0,5.0,2.0,75.0,2017-06-23,2017-06-23
8,1008.0,cisrob2,1396.0,1.0,1.0,5.0,98.0,86.0,2017-06-23,2017-06-23
9,1009.0,cisuabg7,1313.0,1.0,1.0,5.0,2.0,54.0,2017-06-26,2017-06-26


In [15]:
tabledata = {
'Question': ['Age', 'Gender', 'Race', 'Ethnicity', 'Hoehn & Yahr stage 1-3'],
'Variable Name': ['Age', 'Gender', 'Race', 'Ethnicity', 'Hoehn & Yahr stage 1-3'],
'Variable details': ['Integer',
                     'One of: (\'Male\',\'Female\')',
                     'One of: (\'American Indian or Alaska Native\',\'Asian\',\'Black or Afican American\',\'Native Hawaiian or Other Pacific Islander\',\'White\',\'Unknown\')',
                     'One of: (\'Hispanic or Latino\',\'Not Hispanic or Latino\',\'Unknown\')',
                     'One of: (\'no\',\'yes\')']
}
df = pd.DataFrame(tabledata)

# save table as csv file
tablepath = r'Y:\CIS-PD Study\MJFF Curation\tables'
filename = os.path.join(tablepath, 'tabledemo.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

In [13]:
df

,Question,Variable Name,Variable details
0,Age,Age,Integer
1,Gender,Gender,"One of: ('Male','Female')"
2,Race,Race,"One of: ('American Indian or Alaska Native','A..."
3,Ethnicity,Ethnicity,"One of: ('Hispanic or Latino','Not Hispanic or..."
4,Hoehn & Yahr stage 1-3,Hoehn & Yahr stage 1-3,"One of: ('no','yes')"


In [5]:
data101 = pd.read_hdf(os.path.join(path,'form101.h5'))
print(data101.head(5))

   SubjectCode  SiteID   VisitNm    Q11     Q12 ConsentDate   Age  Q03  Q04  \
0       1000.0  1313.0  Baseline  A1784  10.3.2  2017-06-15  63.0  Yes  Yes   
1       1001.0  1313.0  Baseline  A1778  10.3.2  2017-06-15  64.0   No  Yes   
2       1002.0  1313.0  Baseline  A1586  10.3.2  2017-06-16  51.0   No  Yes   
3       1003.0  1313.0  Baseline  A1522  10.3.2  2017-06-19  60.0  Yes  Yes   
4       1004.0  1332.0  Baseline  A1660  10.3.2  2017-06-21  52.0  Yes  Yes   

   Q05  Q06  Q07  Q08 Q09 Q10  
0  Yes  Yes  Yes  Yes  No  No  
1  Yes  Yes  Yes  Yes  No  No  
2  Yes  Yes  Yes  Yes  No  No  
3  Yes  Yes  Yes  Yes  No  No  
4  Yes  Yes  Yes  Yes  No  No  


In [6]:
data126 = pd.read_hdf(os.path.join(path,'form126.h5'))
print(data101.head(5))

   SubjectCode  SiteID   VisitNm    Q11     Q12 ConsentDate   Age  Q03  Q04  \
0       1000.0  1313.0  Baseline  A1784  10.3.2  2017-06-15  63.0  Yes  Yes   
1       1001.0  1313.0  Baseline  A1778  10.3.2  2017-06-15  64.0   No  Yes   
2       1002.0  1313.0  Baseline  A1586  10.3.2  2017-06-16  51.0   No  Yes   
3       1003.0  1313.0  Baseline  A1522  10.3.2  2017-06-19  60.0  Yes  Yes   
4       1004.0  1332.0  Baseline  A1660  10.3.2  2017-06-21  52.0  Yes  Yes   

   Q05  Q06  Q07  Q08 Q09 Q10  
0  Yes  Yes  Yes  Yes  No  No  
1  Yes  Yes  Yes  Yes  No  No  
2  Yes  Yes  Yes  Yes  No  No  
3  Yes  Yes  Yes  Yes  No  No  
4  Yes  Yes  Yes  Yes  No  No  


In [ ]:
data101 = pd.read_hdf(os.path.join(path,'form238.h5'))
print(data101.head(5))

In [ ]:
data101 = pd.read_hdf(os.path.join(path,'form101.h5'))
print(data101.head(5))

# Create Apple watch and phone table
- form101
    - Q11 iphone model
    - Q12 iOS version

In [4]:
path = r'Y:\CIS-PD MUSC\decoded_forms'

In [26]:
tabledata = {
'Question': ['iPhone model to be used by subject', 'iOS version to be used'],
'Variable Name': ['iphone', 'ios'],
'Variable details': ['Text',
                     'Text']
}
df = pd.DataFrame(tabledata)

# save table as csv file
tablepath = r'Y:\CIS-PD Study\MJFF Curation\tables'
filename = os.path.join(tablepath, 'tablespecs.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

In [21]:
# Get questions on forms for Apple watch and phone info
# form101 Q11 and 12 have phone model and iOS version
path = r'Y:\CIS-PD MUSC\decoded_forms'
data101 = pd.read_hdf(os.path.join(path,'form101.h5'))

In [23]:
data101.head(3)

,SubjectCode,SiteID,VisitNm,Q11,Q12,ConsentDate,Age,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10
0,1000.0,1313.0,Baseline,A1784,10.3.2,2017-06-15,63.0,Yes,Yes,Yes,Yes,Yes,Yes,No,No
1,1001.0,1313.0,Baseline,A1778,10.3.2,2017-06-15,64.0,No,Yes,Yes,Yes,Yes,Yes,No,No
2,1002.0,1313.0,Baseline,A1586,10.3.2,2017-06-16,51.0,No,Yes,Yes,Yes,Yes,Yes,No,No


In [8]:
# items: gender, race, ethnicity, age
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\subjenrollment.sas7bdat'
subjenroll = pd.read_sas(filename, format = 'sas7bdat', encoding='iso-8859-1')
subjenroll.columns = subjenroll.columns.str.replace('z', '')
print(subjenroll.head(5))

   SubjectCode FoxInsightID  SiteID  ConsentBeforeEnrollment  Gender  Race  \
0       1000.0     cisuaba1  1313.0                      1.0     1.0   5.0   
1       1001.0     cisuabb2  1313.0                      1.0     1.0   5.0   
2       1002.0     cisuabc3  1313.0                      1.0     1.0   5.0   
3       1003.0     cisuabd4  1313.0                      1.0     2.0   3.0   
4       1004.0      cisnwa1  1332.0                      1.0     1.0   5.0   

   Ethnicity   Age  ConsentDt EnrollmentDt  
0        2.0  63.0 2017-06-15   2017-06-15  
1        2.0  64.0 2017-06-15   2017-06-15  
2        2.0  51.0 2017-06-16   2017-06-16  
3        2.0  60.0 2017-06-19   2017-06-19  
4        2.0  52.0 2017-06-21   2017-06-21  


# Create Table 4 for task and timestamp

In [43]:
filepath = r'Y:\CIS-PD Study\MJFF Curation'
taskdf = pd.read_csv(os.path.join(filepath,'task_timestamp.csv'))

In [75]:
tabledata = {
'Question': ['Subject ID'] + taskdf.columns.tolist()[1:],
'Variable Name': taskdf.columns.tolist(),
'Variable details': ['Integer',
                    'One of: (\'' + '\', \''.join(taskdf.Visit.unique().tolist()) + '\')',
                    'One of: (\'' + '\', \''.join(taskdf.Task.unique().tolist()) + '\')',
                    'Date Time',
                    'Date Time']
}
df = pd.DataFrame(tabledata)

# save table as csv file
tablepath = r'Y:\CIS-PD Study\MJFF Curation\tables'
filename = os.path.join(tablepath, 'summarytable_task_timestamp.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',', index=False)

In [76]:
df

,Question,Variable Name,Variable details
0,Subject ID,SubjID,Integer
1,Visit,Visit,"One of: ('2 Weeks: Time 0', '2 Weeks: Time 30'..."
2,Task,Task,One of: ('Taking a glass of water and drinking...
3,Start Timestamp (UTC),Start Timestamp (UTC),Date Time
4,Stop Timestamp (UTC),Stop Timestamp (UTC),Date Time


# Create Table 6 for sensor info

In [ ]:
'Variable details': ['Integer',
                     'One of: (\'Male\',\'Female\')',
                     'One of: (\'American Indian or Alaska Native\',\'Asian\',\'Black or Afican American\',\'Native Hawaiian or Other Pacific Islander\',\'White\',\'Unknown\')',
                     'One of: (\'Hispanic or Latino\',\'Not Hispanic or Latino\',\'Unknown\')',
                     'One of: (\'no\',\'yes\')']

In [77]:
filepath = r'Y:\CIS-PD Study\MJFF Curation'
sensordf = pd.read_csv(os.path.join(filepath,'sensor_info.csv'))

In [80]:
sensordf.columns.tolist()

['Sensor Location',
 'ECG/EMG (KHz)',
 'Accel (Hz)',
 'Gyro (Hz)',
 'Description of Landmarks',
 'X-axis orientation',
 'Y-axis orientation',
 'Z-axis orientation']

In [81]:
sensordf.head(3)

,Sensor Location,ECG/EMG (KHz),Accel (Hz),Gyro (Hz),Description of Landmarks,X-axis orientation,Y-axis orientation,Z-axis orientation
0,medial_chest,1.0,31.25,NaN,Halfway between base of throat and bottom of s...,Left,Superior,Anterior
1,flexor_digitorum_right,1.0,31.25,NaN,On top of Wrist Flexors,Inferior,Medial,Anterior
2,flexor_digitorum_left,1.0,31.25,NaN,On top of Wrist Flexors,Inferior,Lateral,Anterior


In [82]:
len(sensordf.columns)

8

In [90]:
'One of: (\'' + '\', \''.join(sensordf.iloc[:,0].unique().tolist()) + '\')'

"One of: ('medial_chest', 'flexor_digitorum_right', 'flexor_digitorum_left', 'anterior_thigh_right', 'anterior_thigh_left', 'distal_lateral_shank_right', 'distal_lateral_shank_left', 'sacrum', 'dorsal_hand_right', 'dorsal_hand_left')"

In [97]:
sensordf.iloc[:,1].unique().tolist()

[1.0, nan]

In [98]:
sensordf.iloc[:,2].unique().tolist()

[31.25, 62.5]

In [99]:
sensordf.iloc[:,3].unique().tolist()

[nan, 62.5]

In [100]:
sensordf.iloc[:,4].unique().tolist()

['Halfway between base of throat and bottom of sternum (xiphoid process)',
 'On top of Wrist Flexors',
 'Proximal of Femur Epicondyles',
 'Proximal of Fibular Lateral Malleolus',
 'Superior of Posterior Superior Iliac Spine (PSIS)',
 'Pointing away from thumb, Parallel to wrist joint']

In [101]:
sensordf.iloc[:,5].unique().tolist()

['Left', 'Inferior', 'Right', 'Lateral']

In [102]:
sensordf.iloc[:,6].unique().tolist()

['Superior', 'Medial', 'Lateral', 'Anterior', 'Posterior', 'Inferior']

In [103]:
sensordf.iloc[:,7].unique().tolist()

['Anterior', 'Lateral', 'Posterior']

In [ ]:
31.25, 62.5

In [105]:
tabledata = {
'Question': sensordf.columns.tolist(),
'Variable Name': sensordf.columns.tolist(),
'Variable details': ['One of: (\'' + '\', \''.join(sensordf.iloc[:,0].unique().tolist()) + '\')',
                     'One of: (\'1.0\',\'\')',
                     'One of: (\'31.25\',\'62.5\')',
                     'One of: (\'62.5\',\'\')',
                     'One of: (\'' + '\', \''.join(sensordf.iloc[:,4].unique().tolist()) + '\')',
                     'One of: (\'' + '\', \''.join(sensordf.iloc[:,5].unique().tolist()) + '\')',
                     'One of: (\'' + '\', \''.join(sensordf.iloc[:,6].unique().tolist()) + '\')',
                     'One of: (\'' + '\', \''.join(sensordf.iloc[:,7].unique().tolist()) + '\')']
}
df = pd.DataFrame(tabledata)

# save table as csv file
tablepath = r'Y:\CIS-PD Study\MJFF Curation\tables'
filename = os.path.join(tablepath, 'summarytable_sensorinfo.csv')
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',', index=False)

In [106]:
df

,Question,Variable Name,Variable details
0,Sensor Location,Sensor Location,"One of: ('medial_chest', 'flexor_digitorum_rig..."
1,ECG/EMG (KHz),ECG/EMG (KHz),"One of: ('1.0','')"
2,Accel (Hz),Accel (Hz),"One of: ('31.25','62.5')"
3,Gyro (Hz),Gyro (Hz),"One of: ('62.5','')"
4,Description of Landmarks,Description of Landmarks,One of: ('Halfway between base of throat and b...
5,X-axis orientation,X-axis orientation,"One of: ('Left', 'Inferior', 'Right', 'Lateral')"
6,Y-axis orientation,Y-axis orientation,"One of: ('Superior', 'Medial', 'Lateral', 'Ant..."
7,Z-axis orientation,Z-axis orientation,"One of: ('Anterior', 'Lateral', 'Posterior')"


# Create Table 7 for sensor location and serial number

# change all summary table names - 'summarytable_'....

# Extract data for Methods section
- number of subjects
- subjects per site
- removed subjects

In [27]:
data101.head(3)

,SubjectCode,SiteID,VisitNm,Q11,Q12,ConsentDate,Age,Q03,Q04,Q05,Q06,Q07,Q08,Q09,Q10
0,1000.0,1313.0,Baseline,A1784,10.3.2,2017-06-15,63.0,Yes,Yes,Yes,Yes,Yes,Yes,No,No
1,1001.0,1313.0,Baseline,A1778,10.3.2,2017-06-15,64.0,No,Yes,Yes,Yes,Yes,Yes,No,No
2,1002.0,1313.0,Baseline,A1586,10.3.2,2017-06-16,51.0,No,Yes,Yes,Yes,Yes,Yes,No,No


In [ ]:
# data101.groupby(pd.cut(Pxx.index, bins=binedges)).mean().fillna(0)

In [28]:
data101.groupby(by=data101.SubjectCode)

In [34]:
pd.unique(data101.SubjectCode.values)

array([1000., 1001., 1002., 1003., 1004., 1005., 1006., 1007., 1008.,
       1009., 1010., 1011., 1013., 1014., 1015., 1016., 1017., 1018.,
       1019., 1020., 1021., 1022., 1023., 1024., 1025., 1026., 1027.,
       1028., 1029., 1030., 1031., 1032., 1033., 1034., 1035., 1036.,
       1037., 1038., 1039., 1040., 1041., 1042., 1043., 1044., 1045.,
       1046., 1047., 1048., 1049., 1050., 1051., 1052., 1053., 1054.,
       1055., 1056.])

### total number of patients - DONE

In [32]:
len(pd.unique(data101.SubjectCode.values))

56

In [33]:
len(data101.SubjectCode.values)

56

### check 4 sites - PASS

In [37]:
# checks out
sites = pd.unique(data101.SiteID.values)
print(sites)

[1313. 1332. 1396. 1018.]


### check n for each site - DONE

In [ ]:
data101.loc[data101.SiteID == 1313]

In [40]:
# n for site 1313
len(data101.loc[data101.SiteID == 1313])

13

In [ ]:
data101.loc[data101.SiteID == 1332]

In [41]:
len(data101.loc[data101.SiteID == 1332])

21

In [ ]:
data101.loc[data101.SiteID == 1396]

In [42]:
len(data101.loc[data101.SiteID == 1396])

12

In [ ]:
data101.loc[data101.SiteID == 1018]

In [43]:
len(data101.loc[data101.SiteID == 1018])

10

### Decode site code - DONE

Determine site from subjenrollment.sas7bdat using SiteID and FoxInsightID that has the initials of the institution.
Site 1313 - University of Alabama
Site 1332 - Northwestern University
Site 1396 - University of Rochester
Site 1018 - University of Cincinnati 